In [0]:
#ติดตั้ง pyspark และ import library ที่เป็นฟังก์ชัน sql ของ spark
!pip install pyspark py4j --upgrade

from pyspark.sql.functions import *



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [0]:
#ติดตั้ง opendatasets
%pip install opendatasets --upgrade

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 9.1 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110686 sha256=42ef603a3baf699ba1401c889ad471863a3afc358b3e7c5e0392e69578b6666e
  Stored in directory: /root/.cache/pip/wheels/43/4b/fb/736478af5e8004810081a06259f9aa2f7c3329fc5d03c2c412
Successfully built kaggle
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
#import library opendatasets แล้วใช้คคำสั่ง .download เพื่อโหลดไฟล์ data จาก kaggle
import opendatasets as od

od.download("https://www.kaggle.com/datasets/gopalkalpnde/nyc-tlc-data/data","/dbfs/FileStore/mypath/")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  panuwatsunajan

Your Kaggle Key:

  [REDACTED]

100%|██████████| 1.20G/1.20G [00:46<00:00, 27.4MB/s]


In [0]:
#ใช้ Is เพื่อดูว่าไฟล์ที่โหลดเก็บไฟล์อะไรไว้บ้าง
%sh ls /dbfs/FileStore/mypath/nyc-tlc-data

yellow_tripdata_2015-01.csv
yellow_tripdata_2016-01.csv
yellow_tripdata_2017-01.csv
yellow_tripdata_2018-01.csv


In [0]:
#เปิดไฟล์จากที่อยู่ไฟล์ที่ต้องการ
file_path = "file:/dbfs/FileStore/mypath/nyc-tlc-data/yellow_tripdata_2016-01.csv"

df_1 = spark.read.options(header="true",inferschema = "true").csv(file_path)

df_1.show()

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       2| 2016-01-01 00:00:00|  2016-01-01 00:00:00|              2|          1.1|-73.99037170410156| 40.73469543457031|         1|    

In [0]:
#แปลงข้อมูลจากคอลัมน์ tpep_pickup_datetime ให้เป็น เวลา ชื่อวันและชั่วโมง แล้วเพิ่มในคอลัมน์ใหม่
from pyspark.sql.functions import *

df_1 = df_1.withColumn("Time",date_format(col("tpep_pickup_datetime"),"HH:mm:ss"))
df_1 = df_1.withColumn("DayName", date_format(col("tpep_pickup_datetime"), "EEEE"))
df_1 = df_1.withColumn("Hour", hour(col("tpep_pickup_datetime")))

In [0]:
#สร้างฟังก์ชัน def ที่รับค่าละติจูดและลองติจูด แล้ว return ค่าเป็นชื่อถนนเป็น string type แล้วใช้ฟังก์ชัน def ที่สร้างหาชื่อถนนจากคอลัมน์ pickup_latitude pickup_longitude dropoff_latitude และ dropoff_longitude เพื่อหาชื่อถนนของจุด pick up และ drop off แล้วเก็บไว้ที่คอลัมน์ใหม่
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# สร้างฟังก์ชัน UDF เพื่อค้นหาชื่อถนนจากละติจูดและลองติจูด
@udf(StringType())
def get_street_name(latitude, longitude):
    import requests
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}"
    response = requests.get(url)
    data = response.json()
    if 'address' in data:
        if 'road' in data['address']:
            return data['address']['road']
    return "ไม่พบข้อมูลถนน"

df_1 = df_1.withColumn("street_name_of_pickup", get_street_name(col("pickup_latitude"), col("pickup_longitude")))
df_1 = df_1.withColumn("street_name_of_dropoff", get_street_name(col("dropoff_latitude"), col("dropoff_longitude")))

In [0]:
#เช็คข้อมูลที่ได้จาก df_1
display(df_1.limit(2))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Time,DayName,Hour,street_name_of_dropoff,street_name_of_pickup
2,2016-01-01T00:00:00.000+0000,2016-01-01T00:00:00.000+0000,2,1.1,-73.99037170410156,40.73469543457031,1,N,-73.98184204101562,40.73240661621094,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8,00:00:00,Friday,0,1st Avenue,Union Square East
2,2016-01-01T00:00:00.000+0000,2016-01-01T00:00:00.000+0000,5,4.9,-73.98078155517578,40.72991180419922,1,N,-73.94447326660156,40.716678619384766,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3,00:00:00,Friday,0,Jackson Street,Avenue A


In [0]:
#เลือกคอลัมน์ที่ใช้ทำ Visualization มาเก็บไว้ที่ตัวแปร df_2
df_2 = df_1.select(df_1.tpep_dropoff_datetime, df_1.passenger_count, df_1.pickup_latitude, df_1.pickup_longitude, 
    df_1.dropoff_latitude, df_1.dropoff_longitude, df_1.Time, df_1.DayName, df_1.Hour, df_1.street_name_of_pickup, df_1.street_name_of_dropoff)

In [0]:
#เช็คข้อมูลที่ได้จาก df_2
display(df_2.limit(5))

tpep_dropoff_datetime,passenger_count,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,Time,DayName,Hour,street_name_of_pickup,street_name_of_dropoff
2016-01-01T00:00:00.000+0000,2,40.73469543457031,-73.99037170410156,40.73240661621094,-73.98184204101562,00:00:00,Friday,0,Union Square East,1st Avenue
2016-01-01T00:00:00.000+0000,5,40.72991180419922,-73.98078155517578,40.716678619384766,-73.94447326660156,00:00:00,Friday,0,Avenue A,Jackson Street
2016-01-01T00:00:00.000+0000,1,40.6795654296875,-73.98455047607422,40.78892517089844,-73.95027160644531,00:00:00,Friday,0,Degraw Street,East 100th Street
2016-01-01T00:00:00.000+0000,1,40.718990325927734,-73.99346923828125,40.65733337402344,-73.96224212646484,00:00:00,Friday,0,Chrystie Street,Ocean Avenue
2016-01-01T00:00:00.000+0000,3,40.78133010864258,-73.96062469482422,40.758514404296875,-73.97726440429688,00:00:00,Friday,0,5th Avenue,5th Avenue


In [0]:
#เซฟไฟล์ Output
df_2.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/tables/FinalProject.csv")